In [25]:
import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import dayofweek, dayofmonth, col, when, hour
from pyspark.sql.window import Window
from pyspark.sql import functions as F

app_name = "prepare-data-pyspark"
source_file_fmt = "csv" # csv or parquet

spark = (
    SparkSession.builder
        .master("yarn")
        .appName(app_name)
        .getOrCreate()
)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)  # to pretty print pyspark.DataFrame in jupyter

In [26]:
hdfs_host = "rc1a-dataproc-m-0phjwjdfohabk5n0.mdb.yandexcloud.net"
hdfs_src_dir = "/fraud-data-src"
filename = "2019-08-22.txt"
filepath = f"hdfs://{hdfs_host}{hdfs_src_dir}/{filename}"

In [27]:
from pyspark.sql.types import *        

schema = StructType([StructField("transaction_id", LongType(), True),
                     StructField("tx_datetime", TimestampType(), True),
                     StructField("customer_id", LongType(), True),
                     StructField("terminal_id", LongType(), True),
                     StructField("tx_amount", DoubleType(), True),
                     StructField("tx_time_seconds", LongType(), True),
                     StructField("tx_time_days", LongType(), True),
                     StructField("tx_fraud", LongType(), True),
                     StructField("tx_fraud_scenario", LongType(), True)])

In [28]:
if source_file_fmt == "parquet":
    df = spark.read.option("mergeSchema", "true").schema(schema).parquet(filepath)
elif source_file_fmt == "csv":
    df = spark.read.schema(schema).option("comment", "#").option("header", False).csv(filepath)
else:
    raise Exception(f"Unsupported input {source_file_fmt=}")
df.printSchema()

root
 |-- transaction_id: long (nullable = true)
 |-- tx_datetime: timestamp (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- terminal_id: long (nullable = true)
 |-- tx_amount: double (nullable = true)
 |-- tx_time_seconds: long (nullable = true)
 |-- tx_time_days: long (nullable = true)
 |-- tx_fraud: long (nullable = true)
 |-- tx_fraud_scenario: long (nullable = true)



In [29]:
df

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
0,2019-08-22 06:51:03,0,711,70.91,24663,0,0,0
1,2019-08-22 05:10:37,0,0,90.55,18637,0,0,0
2,2019-08-22 19:05:33,0,753,35.38,68733,0,0,0
3,2019-08-22 07:21:33,0,0,80.41,26493,0,0,0
4,2019-08-22 09:06:17,1,981,102.83,32777,0,0,0
5,2019-08-22 18:41:25,3,205,34.2,67285,0,0,0
6,2019-08-22 03:12:21,3,0,47.2,11541,0,0,0
7,2019-08-22 22:36:40,6,809,139.39,81400,0,0,0
8,2019-08-22 17:23:29,7,184,87.24,62609,0,0,0
9,2019-08-22 21:09:37,8,931,61.7,76177,0,0,0


In [22]:
windows_size_in_days = [1, 7, 30]
delay_window = 7
days = lambda i: i * 86400
delay_period = days(delay_window)


def get_customer_spending_behaviour_features(spark_df):
    for window_size in windows_size_in_days:
        windowSpec = Window().partitionBy(['customer_id']).orderBy(col('tx_datetime').cast('timestamp').cast('long')).rangeBetween(-days(window_size), 0)
        spark_df = spark_df.withColumn('customer_id_nb_tx_'+str(window_size)+'day_window', F.count('*').over(windowSpec)).orderBy('tx_datetime')
        spark_df = spark_df.withColumn('customer_id_avg_amount_'+str(window_size)+'day_window', F.mean('tx_amount').over(windowSpec)).orderBy('tx_datetime')

    return spark_df


def get_count_risk_rolling_window(spark_df):
    windowSpec = Window().partitionBy(['terminal_id']).orderBy(col('tx_datetime').cast('timestamp').cast('long')).rangeBetween(-delay_period, 0)
    spark_df = spark_df.withColumn('nb_fraud_delay', F.sum('tx_fraud').over(windowSpec)).orderBy('tx_datetime')
    spark_df = spark_df.withColumn('nb_tx_delay', F.count('tx_fraud').over(windowSpec)).orderBy('tx_datetime')

    for window_size in windows_size_in_days:
        windowSpec = Window().partitionBy(['terminal_id']).orderBy(col('tx_datetime').cast('timestamp').cast('long')).rangeBetween(-days(window_size)-delay_period, 0)
        spark_df = spark_df.withColumn('nb_fraud_delay_window', F.sum('tx_fraud').over(windowSpec)).orderBy('tx_datetime')
        spark_df = spark_df.withColumn('nb_tx_delay_window', F.count('tx_fraud').over(windowSpec)).orderBy('tx_datetime')
        spark_df = spark_df.withColumn('nb_fraud_window', spark_df['nb_fraud_delay_window']-spark_df['tx_fraud_delay'])
        spark_df = spark_df.withColumn('terminal_id_nb_tx_'+str(window_size)+'day_window', spark_df['nb_tx_delay_window']-spark_df['nb_tx_delay'])
        spark_df = spark_df.withColumn('terminal_id_risk_'+str(window_size)+'day_window', spark_df['nb_fraud_window']/spark_df['terminal_id_nb_tx_'+str(window_size)+'day_window'])

    spark_df = spark_df.na.fill(0)
    spark_df = spark_df.drop('nb_fraud_delay', 'nb_tx_delay', 'nb_fraud_delay_window', 'nb_tx_delay_window', 'nb_fraud_window')

    return spark_df


def preprocess_transactions(spark_df):
    spark_df = spark_df.withColumn('tx_during_weekend', dayofweek(spark_df.TX_DATETIME))
    spark_df = spark_df.withColumn('tx_during_weekend', when((spark_df['tx_during_weekend'] == 1) | (spark_df['tx_during_weekend'] == 7), 1).otherwise(0))

    spark_df = spark_df.withColumn('tx_during_night', hour(spark_df.TX_DATETIME))
    spark_df = spark_df.withColumn('tx_during_night', when(spark_df['tx_during_night'] <= 6, 1).otherwise(0))

    spark_df = get_customer_spending_behaviour_features(spark_df)
    spark_df = get_count_risk_rolling_window(spark_df)

    return spark_df

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario,tx_during_weekend
0,2019-08-22 06:51:03,0,711,70.91,24663,0,0,0,0
1,2019-08-22 05:10:37,0,0,90.55,18637,0,0,0,0
2,2019-08-22 19:05:33,0,753,35.38,68733,0,0,0,0
3,2019-08-22 07:21:33,0,0,80.41,26493,0,0,0,0
4,2019-08-22 09:06:17,1,981,102.83,32777,0,0,0,0
5,2019-08-22 18:41:25,3,205,34.2,67285,0,0,0,0
6,2019-08-22 03:12:21,3,0,47.2,11541,0,0,0,0
7,2019-08-22 22:36:40,6,809,139.39,81400,0,0,0,0
8,2019-08-22 17:23:29,7,184,87.24,62609,0,0,0,0
9,2019-08-22 21:09:37,8,931,61.7,76177,0,0,0,0
